# Westminster Dog Show Data
## Part 3: merging BIS, groups cleaned df with `WKC_merged`

In [3]:
#lib import
import pandas as pd
import numpy as np
import pprint as pp
from tqdm import tqdm
from pathlib import Path

In [10]:
#df import

ModuleNotFoundError: No module named 'WKC_BIS'